<a href="https://colab.research.google.com/github/ikanx101/AppSheet-Sales/blob/main/AppSheet_CALL_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _AppSheet Converter_ 2025 untuk __CALL__

_Converter_ ini dibuat untuk mengubah data hasil _input_ __AppSheet: Call__ ke bentuk tabular yang lebih _excel-user-friendly_.


__Cara pakai:__

1. Klik tombol `Connect` pada pojok kanan atas untuk menyalakan _server cloud Google Colab_.
1. _Upload_ dua _files_ yang dibutuhkan, yakni:
    - Data __AppSheet: Call__. Nama _sheet_ tempat data yang hendak dikonversi: `Merchandise`. Diharapkan penamaan _sheet_ ini __tidak berubah__.
    - Data _master item_.
1. Klik `Runtime` >> `Run all`.
1. Tunggu hingga proses selesai (sampai muncul pesan `PROSES SELESAI`). Kira-kira akan memakan waktu sekitar 5 menit.
1. Silakan _download file_ hasil konversi.

__Keterangan__: Nama _files_ yang di-_upload_ bebas tidak ditentukan tapi nama _sheet_ yang hendak dikonversi harus bernama `Merchandise`.

Catatan: Karena yang kita gunakan saat ini adalah _server_ gratisnya Google, jadi kita harus selalu _install_ beberapa _packages_ setiap kali _run_ skrip ini. Konsekuensinya _runtime_-nya lebih lama.

_Last Update_: 17 Juni 2025

In [ ]:
                #@title Masukkan Nama Files

rm(list=ls())
gc()

nama_file_call   <- "Call.xlsx" #@param {type:"string"}
nama_file_master <- "Master Item.xlsx" #@param {type:"string"}

if(!require(readxl)){
  install.packages("readxl")
  library(readxl)
}
if(!require(reshape2)){
  install.packages("reshape2")
  library(reshape2)
}
if(!require(janitor)){
  install.packages("janitor")
  library(janitor)
}
if(!require(openxlsx)){
  install.packages("openxlsx")
  library(openxlsx)
}
if(!require(expss)){
  install.packages("expss")
  library(expss)
}

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658707,35.2,1454352,77.7,1402828,75.0
Vcells,1227311,9.4,8388608,64.0,1975093,15.1


Loading required package: readxl

Loading required package: reshape2

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘reshape2’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plyr’


Loading required package: janitor

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘janitor’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


Loading required package: openxlsx

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘openxlsx’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘l

In [ ]:
# @title
# libraries yang dibutuhkan
library(dplyr)
library(tidyr)
library(openxlsx)
library(janitor)
library(readxl)
library(expss)

# ==============================================================================
# kita mengambil master data item
# ===========================================================
# converter pertama
 # extract data target utama
  sht  = "Merchandise"
  df   = read_excel(nama_file_call,
                    sheet = sht,col_types = "text")

  nama_kol    = colnames(df)
  batas_akhir = which(nama_kol == "Durasi")

  # kita save dulu variabel yang baru
  df_baru =
    df %>%
    select(`ID Merch`,`Firestart NS`,`Firestart Hilo`,`Status Sekolah`) %>%
    mutate(`Tipe Transaksi` = "AV") %>%
    relocate(`Tipe Transaksi`,.after = `ID Merch`)

  id_penting  = 1:batas_akhir
  kol_penting = nama_kol[id_penting]

  df_1 = df |> select(all_of(kol_penting))

  kol_penting = nama_kol[2:batas_akhir]
  df_2 =
    df |>
    select(-all_of(kol_penting)) |>
    reshape2::melt(id.vars = "ID Merch") |>
    filter(!grepl("sku|tier|fire|status",variable,ignore.case = T)) |>
    mutate(variable = as.character(variable)) |>
    rename("Nama Item" = variable,
           Status      = value)

  # ambil master item
  df_master =
    read_excel(nama_file_master) |>
    janitor::clean_names() |>
    select(item_group,category,brand) |>
    rename("Nama Item" = item_group,
           Brand       = brand,
           Category    = category)

  df_3 = df_2 |> merge(df_master,all.x = T)

  # ini adalah df_final nya
  df_final =
    df_1 |>
    merge(df_3) |>
    relocate(Brand,.after = "Durasi") |>
    relocate(Category,.after = "Brand") |>
    relocate(`Nama Item`,.after = "Category") |>
    # revisi 7 maret
    filter(!Status %in% c("Tidak Jual")) |>
    filter(Status != "<NA>")

  df_final = df_final |> merge(df_baru)

  df_final$Tanggal = as.Date(as.numeric(df_final$Tanggal),
                             origin = "1900-01-01")
  df_final$Tanggal = df_final$Tanggal - 2

  # sekarang kita akan pecah ya
  marker      = nrow(df_final)
  batas_pisah = 10^6

  if(marker < batas_pisah){
    output = list(df_final)
  }
  if(marker > batas_pisah){
    output_1 = df_final[1:batas_pisah,]
    output_2 = df_final[(batas_pisah + 1):marker,]
    output   = list(output_1,output_2)
  }

data_jatim = output
wb <- createWorkbook()

for(ikanx in 1:length(data_jatim)){
  sh = addWorksheet(wb, paste0("Sheet ",ikanx))
  xl_write(data_jatim[ikanx], wb, sh)
}

# Menyimpan workbook ke file
saveWorkbook(wb, file = "Appsheet_converted.xlsx",overwrite = T)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:expss’:

    compute, contains, na_if, recode, vars, where


The following objects are masked from ‘package:maditr’:

    between, coalesce, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘tidyr’


The following objects are masked from ‘package:expss’:

    contains, nest


The following object is masked from ‘package:reshape2’:

    smiths




In [ ]:
#@title Status
print("PROSES SELESAI")
print("file hasil konversi tersimpan dalam nama: Apsheet_converted.xlsx")

[1] "PROSES SELESAI"
[1] "file hasil konversi tersimpan dalam nama: Apsheet_converted.xlsx"
